<a href="https://colab.research.google.com/github/ajayrawatsap/object_detection/blob/master/yolo_train_tf_wizyoung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import tensorflow
print('python version:', sys.version)
print('Tensorflow Version', tensorflow.__version__)

python version: 3.6.8 (default, Oct  7 2019, 12:59:55) 
[GCC 8.3.0]
Tensorflow Version 1.15.0


In [0]:
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [3]:
import os
import matplotlib.pyplot as plt


! git clone https://github.com/ajayrawatsap/YOLOv3_TensorFlow


Cloning into 'YOLOv3_TensorFlow'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 228 (delta 1), reused 0 (delta 0), pack-reused 222
Receiving objects: 100% (228/228), 3.16 MiB | 2.43 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r '/content/drive/My Drive/ML/pothole_detection/train_data.zip' '/content/'
!unzip 'train_data.zip' -d  '/content/'

In [0]:
os.chdir('/content/YOLOv3_TensorFlow')
from voc_to_yolo import convert_voc_yolo

data_path = '/content/train_data/'
annot_txt_path = '/content/YOLOv3_TensorFlow/data/my_data'

convert_voc_yolo(txt_path = os.path.join(annot_txt_path,'train.txt' ),
                 annot_path = os.path.join(data_path,'train_annot' ),
                 img_path = os.path.join(data_path,'train_images' ), 
                 class_names_path = os.path.join(data_path,'class_names.txt' ) 
                 )

convert_voc_yolo(txt_path = os.path.join(annot_txt_path,'val.txt' ),
                 annot_path = os.path.join(data_path,'valid_annot' ),
                 img_path = os.path.join(data_path,'valid_images' ), 
                 class_names_path = os.path.join(data_path,'class_names.txt' ) 
                 )

!cp '/content/train_data/class_names.txt' '/content/YOLOv3_TensorFlow/data/my_data/data.names'
!cp '/content/YOLOv3_TensorFlow/data/my_data/train.txt' '/content/YOLOv3_TensorFlow/train.txt' 

In [7]:


!python get_kmeans.py

anchors_file = os.path.join(annot_txt_path, 'anchors.txt')
f = open(anchors_file, 'w')
f.write("12,5, 14,8, 20,8, 19,13, 26,11, 30,15, 40,19, 30,27, 58,26 \n")
f.close()

anchors are:
12,5, 13,8, 19,8, 24,13, 34,17, 30,27, 45,23, 58,24, 65,37
the average iou is:
0.7822123726135697


In [0]:
!wget https://pjreddie.com/media/files/yolov3.weights  -O  ./data/darknet_weights/yolov3.weights
!python convert_weight.py

## Training


In [0]:
import os
os.chdir('/content/YOLOv3_TensorFlow')

from utils.misc_utils import parse_anchors, read_class_names
from __future__ import division, print_function
import math

class config:
  def __init__(self):
    
    ### Some paths
    self.train_file = './data/my_data/train.txt'  # The path of the training txt file.
    self.val_file = './data/my_data/val.txt'  # The path of the validation txt file.
    self.restore_path = './data/darknet_weights/yolov3.ckpt'  # The path of the weights to restore.
    self.save_dir = './checkpoint/'  # The directory of the weights to save.
    self.log_dir = './data/logs/'  # The directory to store the tensorboard log files.
    self.progress_log_path = './data/progress.log'  # The path to record the training progress.
    self.anchor_path = './data/my_data/anchors.txt'  # The path of the anchor txt file.
    self.class_name_path = './data/my_data/data.names'  # The path of the class names.

    ### Training releated numbers
    self.batch_size = 6
    self.img_size = [416, 416]  # Images will be resized to `img_size` and fed to the network, size format: [width, height]
    self.letterbox_resize = True  # Whether to use the letterbox resize, i.e., keep the original aspect ratio in the resized image.
    self.total_epoches = 100
    self.train_evaluation_step = 100  # Evaluate on the training batch after some steps.
    self.val_evaluation_epoch = 2  # Evaluate on the whole validation dataset after some epochs. Set to None to evaluate every epoch.
    self.save_epoch = 10  # Save the model after some epochs.
    self.batch_norm_decay = 0.99  # decay in bn ops
    self.weight_decay = 5e-4  # l2 weight decay
    self.global_step = 0  # used when resuming training

    ### tf.data parameters
    self.num_threads = 10  # Number of threads for image processing used in tf.data pipeline.
    self.prefetech_buffer = 5  # Prefetech_buffer used in tf.data pipeline.

    ### Learning rate and optimizer
    self.optimizer_name = 'rmsprop' #'momentum'  # Chosen from [sgd, momentum, adam, rmsprop]
    self.save_optimizer = True  # Whether to save the optimizer parameters into the checkpoint file.
    self.learning_rate_init = 1e-4
    self.lr_type = 'piecewise'  # Chosen from [fixed, exponential, cosine_decay, cosine_decay_restart, piecewise]
    self.lr_decay_epoch = 5  # Epochs after which learning rate decays. Int or float. Used when chosen `exponential` and `cosine_decay_restart` lr_type.
    self.lr_decay_factor = 0.96  # The learning rate decay factor. Used when chosen `exponential` lr_type.
    self.lr_lower_bound = 1e-6  # The minimum learning rate.
    # only used in piecewise lr type
    self.pw_boundaries = [30, 50]  # epoch based boundaries
    self.pw_values = [self.learning_rate_init, 3e-5, 1e-5]

    ### Load and finetune
    # Choose the parts you want to restore the weights. List form.
    # restore_include: None, restore_exclude: None  => restore the whole model
    # restore_include: None, restore_exclude: scope  => restore the whole model except `scope`
    # restore_include: scope1, restore_exclude: scope2  => if scope1 contains scope2, restore scope1 and not restore scope2 (scope1 - scope2)
    # choise 1: only restore the darknet body
    # restore_include = ['yolov3/darknet53_body']
    # restore_exclude = None
    # choise 2: restore all layers except the last 3 conv2d layers in 3 scale
    self.restore_include = None
    self.restore_exclude = ['yolov3/yolov3_head/Conv_14', 'yolov3/yolov3_head/Conv_6', 'yolov3/yolov3_head/Conv_22']
    # Choose the parts you want to finetune. List form.
    # Set to None to train the whole model.
    self.update_part = ['yolov3/yolov3_head']

    ### other training strategies
    self.multi_scale_train = True  # Whether to apply multi-scale training strategy. Image size varies from [320, 320] to [640, 640] by default.
    self.use_label_smooth = False #True # Whether to use class label smoothing strategy.
    self.use_focal_loss = True  # Whether to apply focal loss on the conf loss.
    self.use_mix_up = False #True  # Whether to use mix up data augmentation strategy. 
    self.use_warm_up = True  # whether to use warm up strategy to prevent from gradient exploding.
    self.warm_up_epoch = 3  # Warm up training epoches. Set to a larger value if gradient explodes.

    ### some constants in validation
    # nms
    self.nms_threshold = 0.45  # iou threshold in nms operation
    self.score_threshold = 0.3
    # self.score_threshold = 0.01  # threshold of the probability of the classes in nms operation, i.e. score = pred_confs * pred_probs. set lower for higher recall.
    # self.nms_topk = 150  # keep at most nms_topk outputs after nms
    self.nms_topk = 10 #since I have at most 10 objects per image
    # mAP eval
    self.eval_threshold = 0.5  # the iou threshold applied in mAP evaluation
    self.use_voc_07_metric = False  # whether to use voc 2007 evaluation metric, i.e. the 11-point metric

    ### parse some params
    self.anchors = parse_anchors(self.anchor_path)
    self.classes = read_class_names(self.class_name_path)
    self.class_num = len(self.classes)
    self.train_img_cnt = len(open(self.train_file, 'r').readlines())
    self.val_img_cnt = len(open(self.val_file, 'r').readlines())
    self.train_batch_num = int(math.ceil(float(self.train_img_cnt) / self.batch_size))

    self.lr_decay_freq = int(self.train_batch_num * self.lr_decay_epoch)
    self.pw_boundaries = [float(i) * self.train_batch_num + self.global_step for i in self.pw_boundaries]

In [0]:
from __future__ import division, print_function

import tensorflow as tf
import numpy as np
import logging
from tqdm import trange

# import args
args = config()
tf.reset_default_graph()

from utils.data_utils import get_batch_data
from utils.misc_utils import shuffle_and_overwrite, make_summary, config_learning_rate, config_optimizer, AverageMeter
from utils.eval_utils import evaluate_on_cpu, evaluate_on_gpu, get_preds_gpu, voc_eval, parse_gt_rec
from utils.nms_utils import gpu_nms

from model import yolov3

# setting loggers
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s %(message)s',
                    datefmt='%a, %d %b %Y %H:%M:%S', filename=args.progress_log_path, filemode='w')

# setting placeholders
is_training = tf.placeholder(tf.bool, name="phase_train")
handle_flag = tf.placeholder(tf.string, [], name='iterator_handle_flag')
# register the gpu nms operation here for the following evaluation scheme
pred_boxes_flag = tf.placeholder(tf.float32, [1, None, None])
pred_scores_flag = tf.placeholder(tf.float32, [1, None, None])
gpu_nms_op = gpu_nms(pred_boxes_flag, pred_scores_flag, args.class_num, args.nms_topk, args.score_threshold, args.nms_threshold)

##################
# tf.data pipeline
##################
train_dataset = tf.data.TextLineDataset(args.train_file)
train_dataset = train_dataset.shuffle(args.train_img_cnt)
train_dataset = train_dataset.batch(args.batch_size)
train_dataset = train_dataset.map(
    lambda x: tf.py_func(get_batch_data,
                         inp=[x, args.class_num, args.img_size, args.anchors, 'train', args.multi_scale_train, args.use_mix_up, args.letterbox_resize],
                         Tout=[tf.int64, tf.float32, tf.float32, tf.float32, tf.float32]),
    num_parallel_calls=args.num_threads
)
train_dataset = train_dataset.prefetch(args.prefetech_buffer)

val_dataset = tf.data.TextLineDataset(args.val_file)
val_dataset = val_dataset.batch(1)
val_dataset = val_dataset.map(
    lambda x: tf.py_func(get_batch_data,
                         inp=[x, args.class_num, args.img_size, args.anchors, 'val', False, False, args.letterbox_resize],
                         Tout=[tf.int64, tf.float32, tf.float32, tf.float32, tf.float32]),
    num_parallel_calls=args.num_threads
)
val_dataset.prefetch(args.prefetech_buffer)

iterator = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
train_init_op = iterator.make_initializer(train_dataset)
val_init_op = iterator.make_initializer(val_dataset)

# get an element from the chosen dataset iterator
image_ids, image, y_true_13, y_true_26, y_true_52 = iterator.get_next()
y_true = [y_true_13, y_true_26, y_true_52]

# tf.data pipeline will lose the data `static` shape, so we need to set it manually
image_ids.set_shape([None])
image.set_shape([None, None, None, 3])
for y in y_true:
    y.set_shape([None, None, None, None, None])

##################
# Model definition
##################
yolo_model = yolov3(args.class_num, args.anchors, args.use_label_smooth, args.use_focal_loss, args.batch_norm_decay, args.weight_decay, use_static_shape=False)
with tf.variable_scope('yolov3'):
    pred_feature_maps = yolo_model.forward(image, is_training=is_training)
loss = yolo_model.compute_loss(pred_feature_maps, y_true)
y_pred = yolo_model.predict(pred_feature_maps)

l2_loss = tf.losses.get_regularization_loss()

# setting restore parts and vars to update
saver_to_restore = tf.train.Saver(var_list=tf.contrib.framework.get_variables_to_restore(include=args.restore_include, exclude=args.restore_exclude))
update_vars = tf.contrib.framework.get_variables_to_restore(include=args.update_part)

tf.summary.scalar('train_batch_statistics/total_loss', loss[0])
tf.summary.scalar('train_batch_statistics/loss_xy', loss[1])
tf.summary.scalar('train_batch_statistics/loss_wh', loss[2])
tf.summary.scalar('train_batch_statistics/loss_conf', loss[3])
tf.summary.scalar('train_batch_statistics/loss_class', loss[4])
tf.summary.scalar('train_batch_statistics/loss_l2', l2_loss)
tf.summary.scalar('train_batch_statistics/loss_ratio', l2_loss / loss[0])

global_step = tf.Variable(float(args.global_step), trainable=False, collections=[tf.GraphKeys.LOCAL_VARIABLES])
if args.use_warm_up:
    learning_rate = tf.cond(tf.less(global_step, args.train_batch_num * args.warm_up_epoch), 
                            lambda: args.learning_rate_init * global_step / (args.train_batch_num * args.warm_up_epoch),
                            lambda: config_learning_rate(args, global_step - args.train_batch_num * args.warm_up_epoch))
else:
    learning_rate = config_learning_rate(args, global_step)
tf.summary.scalar('learning_rate', learning_rate)

if not args.save_optimizer:
    saver_to_save = tf.train.Saver()
    saver_best = tf.train.Saver()

optimizer = config_optimizer(args.optimizer_name, learning_rate)

# set dependencies for BN ops
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    # train_op = optimizer.minimize(loss[0] + l2_loss, var_list=update_vars, global_step=global_step)
    # apply gradient clip to avoid gradient exploding
    gvs = optimizer.compute_gradients(loss[0] + l2_loss, var_list=update_vars)
    clip_grad_var = [gv if gv[0] is None else [
          tf.clip_by_norm(gv[0], 100.), gv[1]] for gv in gvs]
    train_op = optimizer.apply_gradients(clip_grad_var, global_step=global_step)

if args.save_optimizer:
    print('Saving optimizer parameters to checkpoint! Remember to restore the global_step in the fine-tuning afterwards.')
    saver_to_save = tf.train.Saver()
    saver_best = tf.train.Saver()

with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
    saver_to_restore.restore(sess, args.restore_path)
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter(args.log_dir, sess.graph)

    print('\n----------- start to train -----------\n')

    best_mAP = -np.Inf

    for epoch in range(args.total_epoches):

        sess.run(train_init_op)
        loss_total, loss_xy, loss_wh, loss_conf, loss_class = AverageMeter(), AverageMeter(), AverageMeter(), AverageMeter(), AverageMeter()

        for i in trange(args.train_batch_num):
            _, summary, __y_pred, __y_true, __loss, __global_step, __lr = sess.run(
                [train_op, merged, y_pred, y_true, loss, global_step, learning_rate],
                feed_dict={is_training: True})

            writer.add_summary(summary, global_step=__global_step)

            loss_total.update(__loss[0], len(__y_pred[0]))
            loss_xy.update(__loss[1], len(__y_pred[0]))
            loss_wh.update(__loss[2], len(__y_pred[0]))
            loss_conf.update(__loss[3], len(__y_pred[0]))
            loss_class.update(__loss[4], len(__y_pred[0]))

            if __global_step % args.train_evaluation_step == 0 and __global_step > 0:
                # recall, precision = evaluate_on_cpu(__y_pred, __y_true, args.class_num, args.nms_topk, args.score_threshold, args.nms_threshold)
                recall, precision = evaluate_on_gpu(sess, gpu_nms_op, pred_boxes_flag, pred_scores_flag, __y_pred, __y_true, args.class_num, args.nms_threshold)

                info = "Epoch: {}, global_step: {} | loss: total: {:.2f}, xy: {:.2f}, wh: {:.2f}, conf: {:.2f}, class: {:.2f} | ".format(
                        epoch, int(__global_step), loss_total.average, loss_xy.average, loss_wh.average, loss_conf.average, loss_class.average)
                info += 'Last batch: rec: {:.3f}, prec: {:.3f} | lr: {:.5g}'.format(recall, precision, __lr)
                print(info)
                logging.info(info)

                writer.add_summary(make_summary('evaluation/train_batch_recall', recall), global_step=__global_step)
                writer.add_summary(make_summary('evaluation/train_batch_precision', precision), global_step=__global_step)

                if np.isnan(loss_total.average):
                    print('****' * 10)
                    raise ArithmeticError(
                        'Gradient exploded! Please train again and you may need modify some parameters.')

        # NOTE: this is just demo. You can set the conditions when to save the weights.
        if epoch % args.save_epoch == 0 and epoch > 0:
            if loss_total.average <= 2.:
                saver_to_save.save(sess, args.save_dir + 'model-epoch_{}_step_{}_loss_{:.4f}_lr_{:.5g}'.format(epoch, int(__global_step), loss_total.average, __lr))

        # switch to validation dataset for evaluation
        if epoch % args.val_evaluation_epoch == 0 and epoch >= args.warm_up_epoch:
            sess.run(val_init_op)

            val_loss_total, val_loss_xy, val_loss_wh, val_loss_conf, val_loss_class = \
                AverageMeter(), AverageMeter(), AverageMeter(), AverageMeter(), AverageMeter()

            val_preds = []

            for j in trange(args.val_img_cnt):
                __image_ids, __y_pred, __loss = sess.run([image_ids, y_pred, loss],
                                                         feed_dict={is_training: False})
                pred_content = get_preds_gpu(sess, gpu_nms_op, pred_boxes_flag, pred_scores_flag, __image_ids, __y_pred)
                val_preds.extend(pred_content)
                val_loss_total.update(__loss[0])
                val_loss_xy.update(__loss[1])
                val_loss_wh.update(__loss[2])
                val_loss_conf.update(__loss[3])
                val_loss_class.update(__loss[4])

            # calc mAP
            rec_total, prec_total, ap_total = AverageMeter(), AverageMeter(), AverageMeter()
            gt_dict = parse_gt_rec(args.val_file, args.img_size, args.letterbox_resize)

            info = '======> Epoch: {}, global_step: {}, lr: {:.6g} <======\n'.format(epoch, __global_step, __lr)

            for ii in range(args.class_num):
                npos, nd, rec, prec, ap = voc_eval(gt_dict, val_preds, ii, iou_thres=args.eval_threshold, use_07_metric=args.use_voc_07_metric)
                info += 'EVAL: Class {}: Recall: {:.4f}, Precision: {:.4f}, AP: {:.4f}\n'.format(ii, rec, prec, ap)
                rec_total.update(rec, npos)
                prec_total.update(prec, nd)
                ap_total.update(ap, 1)

            mAP = ap_total.average
            info += 'EVAL: Recall: {:.4f}, Precison: {:.4f}, mAP: {:.4f}\n'.format(rec_total.average, prec_total.average, mAP)
            info += 'EVAL: loss: total: {:.2f}, xy: {:.2f}, wh: {:.2f}, conf: {:.2f}, class: {:.2f}\n'.format(
                val_loss_total.average, val_loss_xy.average, val_loss_wh.average, val_loss_conf.average, val_loss_class.average)
            print(info)
            logging.info(info)

            if mAP > best_mAP:
                best_mAP = mAP
                saver_best.save(sess, args.save_dir + 'best_model_Epoch_{}_step_{}_mAP_{:.4f}_loss_{:.4f}_lr_{:.7g}'.format(
                                   epoch, int(__global_step), best_mAP, val_loss_total.average, __lr))

            writer.add_summary(make_summary('evaluation/val_mAP', mAP), global_step=epoch)
            writer.add_summary(make_summary('evaluation/val_recall', rec_total.average), global_step=epoch)
            writer.add_summary(make_summary('evaluation/val_precision', prec_total.average), global_step=epoch)
            writer.add_summary(make_summary('validation_statistics/total_loss', val_loss_total.average), global_step=epoch)
            writer.add_summary(make_summary('validation_statistics/loss_xy', val_loss_xy.average), global_step=epoch)
            writer.add_summary(make_summary('validation_statistics/loss_wh', val_loss_wh.average), global_step=epoch)
            writer.add_summary(make_summary('validation_statistics/loss_conf', val_loss_conf.average), global_step=epoch)
            writer.add_summary(make_summary('validation_statistics/loss_class', val_loss_class.average), global_step=epoch)

Saving optimizer parameters to checkpoint! Remember to restore the global_step in the fine-tuning afterwards.
INFO:tensorflow:Restoring parameters from ./data/darknet_weights/yolov3.ckpt

----------- start to train -----------



 73%|███████▎  | 27/37 [00:16<00:07,  1.26it/s]

Epoch: 2, global_step: 100 | loss: total: 5.54, xy: 0.67, wh: 1.19, conf: 3.56, class: 0.12 | Last batch: rec: 0.118, prec: 0.182 | lr: 9.009e-05


100%|██████████| 54/54 [00:07<00:00,  7.39it/s]


======> Epoch: 4, global_step: 184.0, lr: 0.0001 <======
EVAL: Class 0: Recall: 0.2185, Precision: 0.0613, AP: 0.0144
EVAL: Recall: 0.2185, Precison: 0.0613, mAP: 0.0144
EVAL: loss: total: 13.21, xy: 0.56, wh: 2.88, conf: 9.63, class: 0.14



...

 70%|███████   | 26/37 [00:15<00:06,  1.73it/s]

Epoch: 75, global_step: 2800 | loss: total: 0.01, xy: 0.01, wh: 0.01, conf: 0.00, class: 0.00 | Last batch: rec: 1.000, prec: 1.000 | lr: 1e-05


  0%|          | 0/37 [00:00<?, ?it/s]

======> Epoch: 76, global_step: 2848.0, lr: 1e-05 <======
EVAL: Class 0: Recall: 0.2101, Precision: 0.5556, AP: 0.1587
EVAL: Recall: 0.2101, Precison: 0.5556, mAP: 0.1587
EVAL: loss: total: 8.74, xy: 0.39, wh: 0.30, conf: 8.03, class: 0.01



 62%|██████▏   | 23/37 [00:13<00:07,  1.76it/s]